In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Introduction

Pretending that we are a Music company that sells music tracks and albums, what are some of the things historical data shows that can be used to guide our future directions?

### Data Overview

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Tracks sold in USA by genre

In [3]:
%%sql
WITH usa_tracks_sold AS
   (
    SELECT il.* 
    FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT 
    g.name genre,
    COUNT(uts.invoice_line_id) tracks_sold,
    ROUND(CAST(COUNT(uts.invoice_line_id) AS Float)/ (
                                        SELECT COUNT(*) FROM usa_tracks_sold
                                        )*100,2) percentage
    FROM usa_tracks_sold uts
    INNER JOIN track t ON uts.track_id = t.track_id
    INNER JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY genre
    ORDER BY tracks_sold DESC;
    


Done.


genre,tracks_sold,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on the sales of tracks of different genres in the USA, we want to focuse on purchasing albums from the 'rock' genre.

### Employees with high sales

In [4]:
%%sql

SELECT 
    e.first_name || ' ' || e.last_name employee_name,
    e.hire_date,
    SUM(i.total) total_sale
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    INNER JOIN employee e ON e.employee_id = c.support_rep_id
    GROUP BY e.employee_id
    ORDER BY 3 DESC;

Done.


employee_name,hire_date,total_sale
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


While it seems like Jane has a better sales performance, it is worth noting that she was hired about half a year eariler than Steve.

### Sales by Country

In [5]:
%%sql

WITH country_or_other AS
(
    SELECT 
    CASE
        WHEN (SELECT COUNT(*) FROM customer WHERE country = c.country) = 1 
        THEN "Other"
        ELSE c.country
    END AS country,
    c.customer_id,
    il.*
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
)

SELECT
country,
total_customers,
total_sales,
average_customer,
average_order
FROM (
    SELECT 
        country,
        COUNT(DISTINCT customer_id) total_customers,
        ROUND(SUM(unit_price),2) total_sales,
        ROUND(SUM(unit_price)/COUNT(DISTINCT customer_id),2) average_customer,
        ROUND(SUM(unit_price)/COUNT(DISTINCT invoice_id),2) average_order,
        CASE WHEN country='Other' THEN 1 ELSE 0 END AS sort
        FROM country_or_other
        GROUP BY country
        ORDER BY sort
    )
ORDER BY 3;

Done.


country,total_customers,total_sales,average_customer,average_order
India,2,183.15,91.58,8.72
Portugal,2,185.13,92.57,6.38
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
Germany,4,334.62,83.66,8.16
France,5,389.07,77.81,7.78
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
USA,13,1040.49,80.04,7.94
Other,15,1094.94,73.0,7.45


It seems like there may be opportunity in Czech Republic since it has a high number of orders per customer. However, it is worth noting that there are not enough data points to support this.

### Buying Albums vs. Invidivial Tracks

In [18]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )
    

SELECT 
album_purchase,
COUNT(album_purchase) invoice_count,
CAST(COUNT(album_purchase) AS Float)/(SELECT COUNT(*) FROM invoice) percentage
FROM(    
    SELECT 
        CASE
            WHEN (
                SELECT t.track_id FROM track t 
                WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                   WHERE t2.track_id = ift.first_track_id)

                EXCEPT

                SELECT il.track_id FROM invoice_line il
                WHERE il.invoice_id = ift.invoice_id
                ) IS NULL

                AND

                (
                SELECT il.track_id FROM invoice_line il
                WHERE il.invoice_id = ift.invoice_id

                EXCEPT

                SELECT t.track_id FROM track t
                WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                    WHERE t2.track_id = ift.first_track_id)
                ) IS NULL
            THEN 'yes'
            ELSE 'no'
            END AS album_purchase
        FROM invoice_first_track ift
    )
GROUP BY 1;

Done.


album_purchase,invoice_count,percentage
no,500,0.8143322475570033
yes,114,0.18566775244299674


While it seems like most people prefer purchasing individual tracks, it still accounts for 18.5% of the revenue, therefore, it is not recommended to only purchase albums.